In [1]:
import sys
sys.path.append('../../../build/python')

import mitsuba as mi
import drjit as dr
import tensorflow as tf

%load_ext autoreload
%autoreload 2
sys.path.append('../drjit')

from interop import wrap, to_drjit, from_drjit, flatten

2024-10-10 12:20:50.344084: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-10 12:20:50.362573: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-10 12:20:50.367423: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-10 12:20:50.379805: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-10 12:20:51.081445: W tensorflow/compiler/tf2

In [63]:
# Test for a single tensor input

@wrap('tf', 'drjit')
def dr_func(input):
    return dr.power(input, 2)

x = tf.constant([1,2,3,4], tf.float32)
with tf.GradientTape() as tape:
    tape.watch(x)
    y = dr_func(x)
grad = tape.gradient(y, x)
print("x:", x)
print("y:", y)
print("Gradient:", grad)  


x: tf.Tensor([1. 2. 3. 4.], shape=(4,), dtype=float32)
y: tf.Tensor([ 1.  4.  9. 16.], shape=(4,), dtype=float32)
Gradient: tf.Tensor([2. 4. 6. 8.], shape=(4,), dtype=float32)


In [64]:
# Test with a list of tensors

@wrap('tf', 'drjit')
def dr_func(inputs):
    result = 0
    for i, input in enumerate(inputs):
        result += dr.power(input, i+1)
    return result

x1 = tf.constant([1, 2, 3, 4], dtype=tf.float32)
x2 = tf.constant([1, 2, 3, 4], dtype=tf.float32)
x3 = tf.constant([1, 2, 3, 4], dtype=tf.float32)
vars = [x1, x2, x3]
with tf.GradientTape() as tape:
    tape.watch(vars)
    result = dr_func(vars)
grad = tape.gradient(result, vars)
print("Result:", result)
print("Gradient:")
for g in grad:
    print(g)

Result: tf.Tensor([ 3. 14. 39. 84.], shape=(4,), dtype=float32)
Gradient:
tf.Tensor([1. 1. 1. 1.], shape=(4,), dtype=float32)
tf.Tensor([2. 4. 6. 8.], shape=(4,), dtype=float32)
tf.Tensor([ 3. 12. 27. 48.], shape=(4,), dtype=float32)


In [65]:
# Test with a nested list of tensors
# Test with a list of tensors

@wrap('tf', 'drjit')
def dr_func(inputs):
    inputs = sum(inputs, [])
    result = 0
    for i, input in enumerate(inputs):
        result += dr.power(input, i+1)
    return result

x1 = tf.constant([1, 2, 3, 4], dtype=tf.float32)
x2 = tf.constant([1, 2, 3, 4], dtype=tf.float32)
x3 = tf.constant([1, 2, 3, 4], dtype=tf.float32)
x4 = tf.constant([1, 2, 3, 4], dtype=tf.float32)
vars = [[x1, x2], [x3, x4]]
with tf.GradientTape() as tape:
    tape.watch(vars)
    result = dr_func(vars)
grad = tape.gradient(result, [x1, [x2, x3, x4]])
print("Result:", result)
print("Gradient:")
for g in grad:
    print(g)


Result: tf.Tensor([  4.  30. 120. 340.], shape=(4,), dtype=float32)
Gradient:
tf.Tensor([1. 1. 1. 1.], shape=(4,), dtype=float32)
[<tf.Tensor: shape=(4,), dtype=float32, numpy=array([2., 4., 6., 8.], dtype=float32)>, <tf.Tensor: shape=(4,), dtype=float32, numpy=array([ 3., 12., 27., 48.], dtype=float32)>, <tf.Tensor: shape=(4,), dtype=float32, numpy=array([  4.,  32., 108., 256.], dtype=float32)>]


In [66]:
# Test with multiple arguments
@wrap('tf', 'drjit')
def dr_func(x1, x2):
    return dr.power(x1, 1) + dr.power(x2, 3)

x1 = tf.constant([1, 2, 3, 4], dtype=tf.float32)
x2 = tf.constant([1, 2, 3, 4], dtype=tf.float32)
with tf.GradientTape() as tape:
    tape.watch([x1, x2])
    result = dr_func(x1, x2)
grad = tape.gradient(result, [x1, x2])
print("Result:", result)
print("Gradient:")
for g in grad:
    print(g)


Result: tf.Tensor([ 2. 10. 30. 68.], shape=(4,), dtype=float32)
Gradient:
tf.Tensor([1. 1. 1. 1.], shape=(4,), dtype=float32)
tf.Tensor([ 3. 12. 27. 48.], shape=(4,), dtype=float32)


In [67]:
# Test with keyword argument
@wrap('tf', 'drjit')
def dr_func(*args, **kwargs):
    result = 0
    for i, x in enumerate(args[0]):
        result += dr.power(x, i+1)

    for i, x in enumerate(kwargs.values()):
        result += 4*x

    return result

x1 = tf.constant([1, 2, 3, 4], dtype=tf.float32)
x2 = tf.constant([2, 3, 4, 5], dtype=tf.float32)
x3 = tf.constant([3, 4, 4, 5], dtype=tf.float32)

with tf.GradientTape() as tape:
    tape.watch([x1, x2, x3])
    result = dr_func([x1, x2], x3=x3)
grad = tape.gradient(result, [x1, x2, x3])
print(grad)

[<tf.Tensor: shape=(4,), dtype=float32, numpy=array([1., 1., 1., 1.], dtype=float32)>, <tf.Tensor: shape=(4,), dtype=float32, numpy=array([ 4.,  6.,  8., 10.], dtype=float32)>, None]


In [70]:
# Test with dict argument
@wrap('tf', 'drjit')
def dr_func(*args, **kwargs):
    result = 0
    for i, x in enumerate(args[0]):
        result += dr.power(x, i+1)

    for i, x in enumerate(args[1].values()):
        result += 4*(i+1)*x

    return result

def tf_func(*args, **kwargs):
    result = 0
    for i, x in enumerate(args[0]):
        result += tf.pow(x, i+1)

    for i, x in enumerate(args[1].values()):
        result += 4*(i+1)*x

    return result

x1 = tf.constant([1, 2, 3, 4], dtype=tf.float32)
x2 = tf.constant([2, 3, 4, 5], dtype=tf.float32)
x3 = tf.constant([3, 4, 4, 5], dtype=tf.float32)

with tf.GradientTape() as tape:
    tape.watch([x1, x2, x3])
    result = dr_func(x1, {'x2' : x2, 'x3': x3})
grad = tape.gradient(result, [x2])
print(result)
for g in grad:
    print(g)

tf.Tensor([320. 332. 336. 348.], shape=(4,), dtype=float32)
tf.Tensor([4. 4. 4. 4.], shape=(4,), dtype=float32)


In [76]:
from tensorflow.autodiff import ForwardAccumulator

In [69]:
# Test with a non differentiable input
@wrap('tf', 'drjit')
def dr_func(x1, x2):
    return dr.power(x1, 1) + dr.power(x2, 1)

def tf_func(x1, x2):
    return tf.pow(x1, 1) + tf.cast(tf.pow(x2, 1), x1.dtype)

x1 = tf.constant([1, 2, 3, 4], dtype=tf.float32)
x2 = tf.constant([1, 2, 3, 4], dtype=tf.int32)
with tf.GradientTape() as tape:
    tape.watch([x1, x2])
    result = dr_func(x1, x2)
grad = tape.gradient(result, [x1, x2])
print("Result:", result)
print("Gradient:")
for g in grad:
    print(g)


Result: tf.Tensor([2. 4. 6. 8.], shape=(4,), dtype=float32)
Gradient:
tf.Tensor([1. 1. 1. 1.], shape=(4,), dtype=float32)
None


In [6]:
import tensorflow as tf
tf.constant([1,2,3,4], tf.float32)

<tf.Tensor: shape=(4,), dtype=float32, numpy=array([1., 2., 3., 4.], dtype=float32)>

In [17]:
import drjit as dr
from drjit.cuda.ad import Float
from drjit.cuda.ad import Int32

t = Int32
is_diff = False

@wrap('drjit', 'torch')
def test_fn(x):
    return x * 2

x = dr.arange(t, 3)
dr.enable_grad(x)
y = test_fn(x)
assert dr.all(y == [0, 2, 4])


if is_diff:
    y.grad = [10, 20, 30]
    dr.backward_to(x)
    assert dr.all(x.grad == [20, 40, 60])
else:
    assert not dr.grad_enabled(y)






In [35]:
from interop import from_drjit, to_drjit

t = dr.llvm.ad.Float
x = dr.arange(t, 3)
z, _ = from_drjit(x, 'tf', True)
y = z*2
z_ = to_drjit(y, 'tf')


In [38]:
import drjit as dr


@wrap('drjit', 'tf')
def test_fn(x):
    return x * 2

t = dr.llvm.ad.Float
x = dr.arange(t, 3)
y = test_fn(x)



RuntimeError: drjit.custom(<interop.WrapADOp>): error while performing a custom differentiable operation. (see above).